In [1]:

from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:2.4.0")
conf.set("spark.mongodb.input.uri", "mongodb://127.0.0.1/test.coll")
conf.set("spark.mongodb.output.uri", "mongodb://127.0.0.1/test.coll")

spark = SparkSession.builder \
    .appName("test-mongo") \
    .master('local[*]') \
    .config(conf=conf) \
    .getOrCreate()


In [7]:

people = spark.createDataFrame([
    ("Bilbo Baggins",  50), ("Gandalf", 1000), ("Thorin", 195),
    ("Balin", 178), ("Kili", 77), ("Dwalin", 169), ("Oin", 167),
    ("Gloin", 158), ("Fili", 82), ("Bombur", None)], ["name", "age"])

people.show()


+-------------+----+
|         name| age|
+-------------+----+
|Bilbo Baggins|  50|
|      Gandalf|1000|
|       Thorin| 195|
|        Balin| 178|
|         Kili|  77|
|       Dwalin| 169|
|          Oin| 167|
|        Gloin| 158|
|         Fili|  82|
|       Bombur|null|
+-------------+----+



In [8]:

print("WRITE/READ")

people.write.format("com.mongodb.spark.sql.DefaultSource").mode("append")\
    .option("database", "people") \
    .option("collection", "contacts") \
    .save()

people.printSchema()

df = spark.read.format("com.mongodb.spark.sql.DefaultSource") \
    .load()

df.show()


WRITE/READ
root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)

+--------------------+----+-------------+
|                 _id| age|         name|
+--------------------+----+-------------+
|[5cc6ac7d78312462...|  82|         Fili|
|[5cc6ac7d78312462...|null|       Bombur|
|[5cc6ac7d78312462...|  50|Bilbo Baggins|
|[5cc6ac7d78312462...| 195|       Thorin|
|[5cc6ac7d78312462...| 158|        Gloin|
|[5cc6ac7d78312462...|1000|      Gandalf|
|[5cc6ac7d78312462...| 167|          Oin|
|[5cc6ac7d78312462...| 178|        Balin|
|[5cc6ac7d78312462...|  77|         Kili|
|[5cc6ac7d78312462...| 169|       Dwalin|
|[5cc6bc4c78312462...| 167|          Oin|
|[5cc6bc4c78312462...|1000|      Gandalf|
|[5cc6bc4c78312462...| 169|       Dwalin|
|[5cc6bc4c78312462...| 195|       Thorin|
|[5cc6bc4c78312462...| 178|        Balin|
|[5cc6bc4c78312462...|  77|         Kili|
|[5cc6bc4c78312462...|  50|Bilbo Baggins|
|[5cc6bc4c78312462...|  82|         Fili|
|[5cc6bc4c78312462...|null|     

In [9]:

print("CHANGE COLLECTION TO people")

df = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
    .option("uri", "mongodb://127.0.0.1/people.contacts")\
    .load()

df.show()


CHANGE COLLECTION
+--------------------+----+-------------+
|                 _id| age|         name|
+--------------------+----+-------------+
|[5cc6bc9578312462...|  82|         Fili|
|[5cc6bc9578312462...|null|       Bombur|
|[5cc6bc9578312462...| 167|          Oin|
|[5cc6bc9578312462...|  50|Bilbo Baggins|
|[5cc6bc9578312462...| 158|        Gloin|
|[5cc6bc9578312462...| 195|       Thorin|
|[5cc6bc9578312462...| 178|        Balin|
|[5cc6bc9578312462...|  77|         Kili|
|[5cc6bc9578312462...|1000|      Gandalf|
|[5cc6bc9578312462...| 169|       Dwalin|
|[5cc6bd1178312462...|1000|      Gandalf|
|[5cc6bd1178312462...|  50|Bilbo Baggins|
|[5cc6bd1178312462...| 178|        Balin|
|[5cc6bd1178312462...|  77|         Kili|
|[5cc6bd1178312462...| 167|          Oin|
|[5cc6bd1178312462...| 158|        Gloin|
|[5cc6bd1178312462...| 169|       Dwalin|
|[5cc6bd1178312462...| 195|       Thorin|
|[5cc6bd1178312462...|  82|         Fili|
|[5cc6bd1178312462...|null|       Bombur|
+---------------

In [3]:

print("AGGREGATION")

pipeline = "{'$match': {'age': { '$gte': 150 }}}"
df = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
    .option("pipeline", pipeline)\
    .load()

df.show()


AGGREGATION
+--------------------+----+-------+
|                 _id| age|   name|
+--------------------+----+-------+
|[5cc6ac7d78312462...| 195| Thorin|
|[5cc6ac7d78312462...| 158|  Gloin|
|[5cc6ac7d78312462...|1000|Gandalf|
|[5cc6ac7d78312462...| 167|    Oin|
|[5cc6ac7d78312462...| 178|  Balin|
|[5cc6ac7d78312462...| 169| Dwalin|
|[5cc6bc4c78312462...| 167|    Oin|
|[5cc6bc4c78312462...|1000|Gandalf|
|[5cc6bc4c78312462...| 169| Dwalin|
|[5cc6bc4c78312462...| 195| Thorin|
|[5cc6bc4c78312462...| 178|  Balin|
|[5cc6bc4c78312462...| 158|  Gloin|
+--------------------+----+-------+



In [5]:

print("FILTER")
df.filter(df['age'] >= 1000)\
    .show()


FILTER
+-------+----+
|   name| age|
+-------+----+
|Gandalf|1000|
|Gandalf|1000|
+-------+----+



In [4]:

print("SQL")

df.createOrReplaceTempView("temp")
df = spark.sql("SELECT name, age FROM temp WHERE name LIKE '%Gandalf%'")
df.show()


SQL
+-------+----+
|   name| age|
+-------+----+
|Gandalf|1000|
|Gandalf|1000|
+-------+----+



In [6]:
spark.stop()
